In [11]:
import numpy as np
import pandas as pd
import re 
import plotly.express as px

In [5]:
df = pd.read_csv('/content/drive/MyDrive/df_top_map.csv')

In [6]:
df

,Unnamed: 0,Unnamed: 0.1,published_date,section,adx_keywords,byline,type,title,abstract,des_facet,org_facet,per_facet,geo_facet
0,0,0,2022-02-24,World,Russian Invasion of Ukraine (2022);Defense and...,By Austin Ramzy,Article,The Invasion of Ukraine: How Russia Attacked a...,Explosions thundered in Ukrainian cities short...,"['Russian Invasion of Ukraine (2022)', 'Defens...","['European Union', 'North Atlantic Treaty Orga...","['Biden, Joseph R Jr', 'Zelensky, Volodymyr', ...","['Eastern Europe', 'Russia', 'Ukraine', 'Europe']"
1,1,1,2022-02-24,World,Russian Invasion of Ukraine (2022);United Stat...,"By David D. Kirkpatrick, Maggie Astor and Cati...",Article,"Trump Praises Putin, Leaving Republicans in a ...","G.O.P. leaders, while condemning Russia’s inva...","['Russian Invasion of Ukraine (2022)', 'United...","['Fox News Channel', 'Republican Party']","['Biden, Joseph R Jr', 'Putin, Vladimir V', 'T...","['Russia', 'Ukraine']"
2,2,2,2022-02-25,Technology,Russian Invasion of Ukraine (2022);Conspiracy ...,By Davey Alba and Stuart A. Thompson,Article,‘I’ll Stand on the Side of Russia’: Pro-Putin ...,After marinating in conspiracy theories and Do...,"['Russian Invasion of Ukraine (2022)', 'Conspi...","['Telegram LLC', 'Twitter', 'Facebook Inc']","['Putin, Vladimir V', 'Trump, Donald J', 'Bide...",[]
3,3,3,2022-02-28,U.S.,Russian Invasion of Ukraine (2022);Cyberwarfar...,"By David E. Sanger, Julian E. Barnes and Kate ...",Article,"As Tanks Rolled Into Ukraine, So Did Malware. ...",After years of talks about the need for public...,"['Russian Invasion of Ukraine (2022)', 'Cyberw...","['Microsoft Corp', 'United States Cyber Comman...",[],"['Russia', 'Ukraine']"
4,4,4,2022-02-27,U.S.,Russian Invasion of Ukraine (2022);Nuclear Wea...,By David E. Sanger and William J. Broad,Article,"Putin Declares a Nuclear Alert, and Biden Seek...",When the Russian leader ordered his nuclear fo...,"['Russian Invasion of Ukraine (2022)', 'Nuclea...",[],"['Biden, Joseph R Jr', 'Putin, Vladimir V']","['Russia', 'Ukraine', 'United States']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,195,15,2022-04-08,Movies,Academy Awards (Oscars);Movies;Actors and Actr...,By Nicole Sperling,Article,Will Smith Barred From Attending Oscars for 10...,"Smith, who won best actor this year, can still...","['Academy Awards (Oscars)', 'Movies', 'Actors ...",['Academy of Motion Picture Arts and Sciences'],"['Smith, Will', 'Rock, Chris', 'Smith, Jada Pi...",[]
195,196,16,2022-04-08,U.S.,"Storming of the US Capitol (Jan, 2021);United ...",By Alan Feuer,Article,Pro-Trump Rally Planner Is Cooperating in Just...,"Ali Alexander, who was a key figure in the “St...","['Storming of the US Capitol (Jan, 2021)', 'Un...",['Justice Department'],"['Alexander, Ali (Ali Abdul Akbar)', 'Trump, D...",[]
196,197,17,2015-01-09,Style,Love (Emotion);Dating and Relationships;Psycho...,By Daniel Jones,Article,The 36 Questions That Lead to Love,A series of personal questions used by the psy...,"['Love (Emotion)', 'Dating and Relationships',...",[],"['Aron, Arthur (1945- )']",[]
197,198,18,2022-04-07,Well,Sleep;Content Type: Service;Coronavirus (2019-...,By Melinda Wenner Moyer,Article,The Trouble With Sleep Aids,Not all sleep medications are the same. Here’s...,"['Sleep', 'Content Type: Service', 'Coronaviru...",[],[],[]


In [7]:
df["geo_facet"] = df["geo_facet"].apply(eval)
df["per_facet"] = df["per_facet"].apply(eval)
df["des_facet"] = df["des_facet"].apply(eval)

In [8]:
def to_1D(series):
    return pd.DataFrame([x for _list in series for x in _list],columns=['kolumna'])

In [23]:
all_countries = to_1D(df['geo_facet'])
all_countries['waga'] = 1
all_countries

#per = to_1D(df['per_facet'])
#per['waga'] = 1
#per.groupby('kolumna').sum()

#des = to_1D(df['des_facet'])
#des['waga'] = 1
#des.groupby('kolumna').sum()

,kolumna,waga
0,Eastern Europe,1
1,Russia,1
2,Ukraine,1
3,Europe,1
4,Russia,1
...,...,...
236,New York State,1
237,Russia,1
238,Ukraine,1
239,Brooklyn (NYC),1


In [31]:
country_short = all_countries[~(all_countries['kolumna'].str.contains('\W'))] #jednowyrazowe bez nawiasów
all_countries[(all_countries['kolumna'].str.contains('\W')) & (~(all_countries['kolumna'].str.contains('\w+ \w+')))] #jednowyrazowe z nawiasem
all_countries[(all_countries['kolumna'].str.contains('[(]')) & (all_countries['kolumna'].str.contains('\w+ \w+'))] #wielowyrazowe z nawiasem
country_long = all_countries[(~(all_countries['kolumna'].str.contains('[(]'))) & (all_countries['kolumna'].str.contains('\w+ \w+'))] #wielowyrazowe bez nawiasów
countries_regex = pd.concat([country_short, country_long])

In [17]:
#country_short.groupby('kolumna').sum()

In [16]:
#country_long.groupby('kolumna').sum()

In [32]:
zgrupowane_all = all_countries.groupby('kolumna').sum() 
zgrupowane_all = zgrupowane_all.reset_index()
zgrupowane_regex = countries_regex.groupby('kolumna').sum() 
zgrupowane_regex = zgrupowane_regex.reset_index()

In [33]:
fig = px.choropleth(zgrupowane_all, locations="kolumna", locationmode='country names', 
                     color="waga", hover_name="kolumna",
                     range_color= [0, max(zgrupowane_all['waga'])+2], 
                     projection="natural earth",  labels={'waga':'Liczba wystąpień'},
                     title='Kraje, których dotyczą artykuły')

fig.show()

In [34]:
fig = px.choropleth(zgrupowane_regex, locations="kolumna", locationmode='country names', 
                     color="waga", hover_name="kolumna",
                     range_color= [0, max(zgrupowane_regex['waga'])+2], 
                     projection="natural earth",  labels={'waga':'Liczba wystąpień'},
                     title='Kraje, których dotyczą artykuły')

fig.show()